The challenge below is to have diff of two numbers, m and n as in $|m-n| = d$ 
If the condition is true, the total state value should be equal to -1 or 1 otherwise. 

https://pennylane.ai/challenges/the_oracle_of_the_exact_distance/

The logic below does.. 
1. Basis embedding of m, n values onto wires_m and wires_n
2. QFT on the wires_aux. 
3. Add m and -n to the wires_aux which produces a diff. 
4. The diff could be negative so use 2s compliment ( flip all bits and add 1 ) to covert to + ve value
5. Check if diff in binary as q state is active, if yes -> introduce phase on the last qubit. 
6. Uncompute the steps 2-5. 


In [99]:
import json
import pennylane as qml
import pennylane.numpy as np

wires_m = [0, 1, 2]  # qubits needed to encode m
wires_n = [3, 4, 5]  # qubits needed to encode n
wires_aux = [6, 7, 8, 9, 10]  # auxiliary qubits you can use
np.set_printoptions(threshold = np.inf)

def add_k_fourier(k, wires, multiply=1):
    for j in range(len(wires)):
        qml.RZ(multiply * k * np.pi / (2**j), wires=wires[j])

def add(wires):
    add_k_fourier(1, wires)
       
def add_m(twires, w, multiply=1):
    for i in range(len(w)):
        qml.ctrl(add_k_fourier, control=w[i])(2 **(len(w) - i - 1), twires, multiply)

def b2s_compliment(twires):
    for i in twires[1:]:
        qml.PauliX(i)
    qml.QFT(wires=twires[1:])
    add_k_fourier(1, wires=twires[1:])
    qml.adjoint(qml.QFT)(wires=twires[1:])

def compute_modulus_diff(d):
    twires_aux = wires_aux[:-1]
    qml.QFT(wires=twires_aux)
    add_m(twires_aux,wires_m,)
    add_m(twires_aux,wires_n,-1)
    qml.adjoint(qml.QFT)(wires=twires_aux)
    qml.ctrl(b2s_compliment, control=[wires_aux[0]], control_values="1")(twires_aux)

def oracle_distance(d):
    compute_modulus_diff(d)
    qml.ctrl(qml.PauliX, control=wires_aux[1:-1], control_values=format(d, "#05b")[2:])(wires_aux[-1])
    qml.adjoint(compute_modulus_diff)(d)
    qml.RY(np.pi, wires_aux[-1])

dev = qml.device("default.qubit.legacy", wires=11)

@qml.qnode(dev)
def circuit(m, n, d):
    qml.BasisEmbedding(features=m, wires=wires_m)
    qml.BasisEmbedding(features=n, wires=wires_n)
    oracle_distance(d)
    return qml.state()


def run(test_case_input: str) -> str:
    outputs = []
    d = int(json.loads(test_case_input))
    for n in range(8):
        for m in range(8):
            s = circuit(m, n, d)
            values = s
            # print (s)
            # for i in range(len(values)):
            #     values [i] = np.round(values[i], 3)
            #     if values[i]:
            #         print ((m,n,d), format(i, '#013b')[2:], i, values[i])
                    
            outputs.append(sum(s).real)
        # break
    # qml.draw_mpl(circuit, decimals=2)(0, 2, 2)

    output_list = [round(elem.numpy()) for elem in outputs]
    return str(output_list)

def check(solution_output: str, expected_output: str) -> None:
    i = 0
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(solution_output, expected_output)
    circuit(np.random.randint(7), np.random.randint(7), np.random.randint(7))
    tape = circuit.qtape
    names = [op.name for op in tape.operations]    
    assert names.count("QubitUnitary") == 0, "Can't use custom-built gates!"

# These are the public test cases
test_cases = [
    ('0', '[-1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1]'),
    ('1', '[1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1]'),
    ('2', '[1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1]'),
    ('3', '[1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1]'),
    ('4', '[1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1]'),
    ('5', '[1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1]'),
    ('6', '[1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1]'),
    ('7', '[1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)
        print (output)
    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '0'...
[-1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1]
Correct!
Running test case 1 with input '1'...
[1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1]
Correct!
Running test case 2 with input '2'...
[1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1]
Correct!
Running test case 3 with input '3'...
[1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1]
Corre